<a href="https://colab.research.google.com/github/ajw1587/Pytorch_Study/blob/main/06_Softmax_Classification_Pytorch_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install torchvision

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

torch.manual_seed(1)

In [11]:
z = torch.FloatTensor([1, 2, 3])

hypothesis = F.softmax(z, dim = 0)
print(hypothesis)
hypothesis.sum()

tensor([0.0900, 0.2447, 0.6652])


tensor(1.)

In [12]:
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [21]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 0, 4])


In [22]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.]])

In [24]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4741, grad_fn=<MeanBackward0>)


In [29]:
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4741, grad_fn=<NllLossBackward>)

In [30]:
F.cross_entropy(z, y)

tensor(1.4741, grad_fn=<NllLossBackward>)

In [31]:
x_train = [[1, 2, 1, 1],
           [2, 1, 2, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)     # 인덱스의 위치이기 때문에 정수

In [36]:
# 모델 초기화
w = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([w, b], lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  # Cost 계산
  hypothesis = F.softmax(x_train.matmul(w) + b, dim = 1)
  y_one_hot = torch.zeros_like(hypothesis)
  y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
  cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim =1))).sum(dim=1).mean()

  # Cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.879320
Epoch  200/1000 Cost: 0.805669
Epoch  300/1000 Cost: 0.769445
Epoch  400/1000 Cost: 0.747767
Epoch  500/1000 Cost: 0.733252
Epoch  600/1000 Cost: 0.722869
Epoch  700/1000 Cost: 0.715099
Epoch  800/1000 Cost: 0.709086
Epoch  900/1000 Cost: 0.704307
Epoch 1000/1000 Cost: 0.700426


In [37]:
# 모델 초기화
w = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([w, b], lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  # Cost 계산
  z = x_train.matmul(w) + b
  cost = F.cross_entropy(z, y_train)

  # Cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.705542
Epoch  200/1000 Cost: 0.618635
Epoch  300/1000 Cost: 0.562743
Epoch  400/1000 Cost: 0.518396
Epoch  500/1000 Cost: 0.479757
Epoch  600/1000 Cost: 0.444251
Epoch  700/1000 Cost: 0.410524
Epoch  800/1000 Cost: 0.377795
Epoch  900/1000 Cost: 0.345596
Epoch 1000/1000 Cost: 0.313665


In [38]:
class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(4, 3)

  def forward(self, x):
    return self.linear(x)

In [39]:
model = SoftmaxClassifierModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  # H(x) 계산
  prediction = model(x_train)

  # Cost 계산
  cost = F.cross_entropy(prediction, y_train)

  # Cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 20번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 1.950346
Epoch  100/1000 Cost: 0.698547
Epoch  200/1000 Cost: 0.598680
Epoch  300/1000 Cost: 0.536322
Epoch  400/1000 Cost: 0.485161
Epoch  500/1000 Cost: 0.438642
Epoch  600/1000 Cost: 0.394191
Epoch  700/1000 Cost: 0.350602
Epoch  800/1000 Cost: 0.307334
Epoch  900/1000 Cost: 0.264584
Epoch 1000/1000 Cost: 0.225920
